https://www.policinginstitute.org/publication/the-minneapolis-domestic-violence-experiment/

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from typing import Dict, List

from dowhy import CausalModel
# statsmodels' instrument variable implementation
from statsmodels.sandbox.regression.gmm import IV2SLS

# Display only double digits
pd.set_option('display.float_format', '{:.2f}'.format)

# Data Cleaning 

In [2]:
TREATMENT_CODE_NAME_MAPPING: Dict[int, str] = {
    1: 'Arrest',
    2: 'Advise',
    3: 'Separate',
}

In [3]:
data = pd.read_stata(
    "../data/mdve.dta"
)


data.head()

,ID,T_RANDOM,MONTH,YEAR,CLOCK,TIME,P_REPORT,CCN,V_RACE,S_RACE,...,P_ASSLT2,P_CALMED,P_REFER,P_ADVISE,T_FINAL,REASON1,REASON2,REASON3,REASON4,R_RELATE
0,372020123,3,5.00,81.00,125.00,35.00,1,83905.00,1,1.00,...,1.00,9.00,9.00,9.00,3,1.00,1.00,1.00,1.00,NaN
1,441050111,1,5.00,81.00,515.00,40.00,1,NaN,1,2.00,...,2.00,9.00,9.00,9.00,1,1.00,1.00,1.00,1.00,NaN
2,281200111,1,5.00,81.00,1910.00,15.00,1,77252.00,3,3.00,...,1.00,9.00,9.00,9.00,1,1.00,1.00,1.00,1.00,NaN
3,681260123,3,7.00,81.00,210.00,NaN,0,NaN,3,3.00,...,1.00,9.00,9.00,9.00,3,1.00,1.00,1.00,1.00,NaN
4,232010123,3,5.00,81.00,100.00,NaN,1,74836.00,1,1.00,...,1.00,9.00,9.00,9.00,3,9.00,1.00,1.00,1.00,NaN


In [4]:
# drop if the actual treatment is "other"
data = data[data['T_FINAL'] != 4]

# coddle
data['z_coddle'] = data['T_RANDOM'].apply(lambda x: 0 if x == 1 else 1)
data['d_coddle'] = data['T_FINAL'].apply(lambda x: 0 if x == 1 else 1)

In [5]:
data.shape

In [6]:
data.columns

Index(['ID', 'T_RANDOM', 'MONTH', 'YEAR', 'CLOCK', 'TIME', 'P_REPORT', 'CCN',
       'V_RACE', 'S_RACE', 'V_CHEM', 'S_CHEM', 'S_DMNOR1', 'S_DMNOR2',
       'WEAPON', 'GUNS', 'P_ASSLT1', 'P_ASSLT2', 'P_CALMED', 'P_REFER',
       'P_ADVISE', 'T_FINAL', 'REASON1', 'REASON2', 'REASON3', 'REASON4',
       'R_RELATE', 'z_coddle', 'd_coddle'],
      dtype='object')

## Calculate Assigned and Delivered Treatments

The table 3.3 in Chapter 3

In [7]:
compliance_data = (
    data
    .groupby(by=['T_RANDOM', 'T_FINAL'])
    .agg({'ID': 'count'})
    .reset_index()
    .pivot_table(
        values='ID',
        index='T_RANDOM',
        columns='T_FINAL'
    )
    .fillna(0)
    .astype(int)
    .rename(
        index=TREATMENT_CODE_NAME_MAPPING,
        columns=TREATMENT_CODE_NAME_MAPPING
    )
)

compliance_data = (
    compliance_data
    .div(compliance_data.sum(axis=1), axis=0)
    * 100
)

compliance_data

T_FINAL,Arrest,Advise,Separate
T_RANDOM,,,
Arrest,98.91,0.00,1.09
Advise,17.59,77.78,4.63
Separate,22.81,4.39,72.81


# Causal Inference

## `DoWhy` Implementation

## `statsmodels` Implementation